#### This file get the data from the GOOGLE PLACE API 

In [ ]:
#import the libraries
import requests
import pandas as pd
import time
import csv
from tqdm import tqdm
import re
from numpy import ones

In [ ]:

#use your google place id
API_KEY = "PUT YOUR API KEY HERE"

In [ ]:
#mount the drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#From the datasets folder on github download the report_eng(Beaches) and load it on Collab
beaches_df = pd.read_csv("/content/report_eng(Beaches).csv", delimiter=";")

In [ ]:
# Extract the coordinates
coordinates = beaches_df[["X","Y"]]

In [ ]:
#check the dataset very quick
beaches_df.head(4)

,Beach code,Beach,X,Y,Orientation (Codice),Orientation,Region,Country
0,40,Aciddara,"13,53734493","38,08036623",SE,Sud-Est,NaN,Italia
1,149,Bagni della Regina Giovanna - Sorrento (Na),"14,35122371","40,63355879",NW,Nord-Ovest,NaN,Italia
2,59,baia delle favole,"9,391254298","44,27072752",W,Ovest,NaN,Italia
3,123,Boschetto Steccato di Cutro,"16,93195671","38,93462782",NaN,NaN,NaN,Italia


In [ ]:

from typing_extensions import NoExtraItems
def convert_coord(coord):

  """This function takes a pair of cords
  and makes it point floating number"""

  try:
    if isinstance(coord,str):
      return float(coord.replace("," ,"."))
    return float(coord)
  except:
      return None


In [ ]:
#call the function to change the coords
beaches_df["lat"] = beaches_df["Y"].apply(convert_coord)
beaches_df["lng"] = beaches_df["X"].apply(convert_coord)

In [ ]:
#check it if from dot changed to comma
beaches_df.head(4)

,Beach code,Beach,X,Y,Orientation (Codice),Orientation,Region,Country,lat,lng
0,40,Aciddara,"13,53734493","38,08036623",SE,Sud-Est,NaN,Italia,38.080366,13.537345
1,149,Bagni della Regina Giovanna - Sorrento (Na),"14,35122371","40,63355879",NW,Nord-Ovest,NaN,Italia,40.633559,14.351224
2,59,baia delle favole,"9,391254298","44,27072752",W,Ovest,NaN,Italia,44.270728,9.391254
3,123,Boschetto Steccato di Cutro,"16,93195671","38,93462782",NaN,NaN,NaN,Italia,38.934628,16.931957


In [ ]:
# Clean beach names for better matching

def clean_name(name):

# This function takes a name word and makes and bring it into a
# specific format

    if pd.isna(name):
        return ""
    name = str(name).lower()
    # Remove extra spaces, special characters
    name = re.sub(r'[^\w\s]', ' ', name)
    name = ' '.join(name.split())
    return name

In [ ]:
# Apply the function to have
beaches_df["clean_beach_name"] = beaches_df["Beach"].apply(clean_name)


In [ ]:
# Check which beaches you should search for
print("Beaches in CSV to search for:")
print(beaches_df[["Beach", "lat", "lng", "clean_beach_name"]].head())

Beaches in CSV to search for:
                                         Beach        lat        lng  \
0                                     Aciddara  38.080366  13.537345   
1  Bagni della Regina Giovanna - Sorrento (Na)  40.633559  14.351224   
2                            baia delle favole  44.270728   9.391254   
3                 Boschetto Steccato di Cutro   38.934628  16.931957   
4                             Cala Santa Maria  38.708912  13.195334   

                          clean_beach_name  
0                                 aciddara  
1  bagni della regina giovanna sorrento na  
2                        baia delle favole  
3              boschetto steccato di cutro  
4                         cala santa maria  


In [ ]:
def get_beaches(lat, lng, radius = 1000):

    """
    This function takes as in iput the latidute and longitude of each beach
    calls the google API key and searches within the radius to find either beaches
    defined as tourist atractions or natural habitat, removing the duplicates
    and returnes them
    """


    all_results = []
    place_types = ["natural_feature", "tourist_attraction"]

    for place_type in place_types:
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            "location": f"{lat},{lng}",
            "radius": radius,
            "type": place_type,  # Single type per request
            "key": API_KEY
        }

        response = requests.get(url, params=params)
        time.sleep(1)  # Add delay between calls

        if response.status_code == 200:
            data = response.json()
            all_results.extend(data.get("results", []))

    # Remove duplicates based on place_id
    unique_results = {place["place_id"]: place for place in all_results}.values()

    return {"results": list(unique_results)}


In [ ]:
def is_valid_beach(place):
    name = place.get("name", "").lower()
    types = place.get("types", [])

    beach_keywords = ["spiaggia", "lido", "beach", "playa", "baia", "cala",
                      "mare", "sea", "sabbia", "sand", "bagni", "stabilimento"]

    commercial_types = ["restaurant", "lodging", "bar", "cafe", "store",
                        "night_club", "food", "meal_takeaway"]

    has_beach_keyword = any(keyword in name for keyword in beach_keywords)
    is_commercial = any(comm_type in types for comm_type in commercial_types)

    # Check if it's specifically a tourist attraction
    is_tourist_attraction = "tourist_attraction" in types

    # A place is a valid beach if:
    # 1. It has beach keywords AND is not purely commercial
    # OR 2. It's marked as tourist_attraction AND has beach-related context
    return (has_beach_keyword and not is_commercial) or \
           (is_tourist_attraction and not is_commercial and has_beach_keyword)

In [ ]:
def extract_beach_info(place):
    """Extract relevant information from a beach place"""
    return {
        "name": place.get("name", ""), # to get the name of the beach
        "rating": place.get("rating", None), # to get the rating from 0 to 5
        "user_ratings_total": place.get("user_ratings_total", 0), # to get the overall user rating
        "lat": place["geometry"]["location"]["lat"], # to get the latitude from google api
        "lng": place["geometry"]["location"]["lng"], # to get the longitude from google api
        "types": ", ".join(place.get("types", [])), # to get the google place category type such as natural_feature for beach
        "vicinity": place.get("vicinity", "") # to get the vicinity area
    }

In [ ]:
def process_coordinates(coordinates_df, radius=2000, delay=2):

  '''
  It takes a table of coordinates and, for each coordinate, searches Google Maps for nearby beaches,
  filters out non-beaches, extracts useful information, and returns a single list of all beaches found
  '''


  all_beaches = []  # empty list
  processed_locations = 0 # the processed locations
  beaches_found = 0

  print(f"processing..  {len(coordinates_df)} coordinates")

  for idx, row in tqdm(coordinates_df.iterrows(), total = len(coordinates_df)):
    # CONVERT COMMAS TO DOTS for decimal places
    lat_str = str(row["Y"]).replace(",", ".")
    lng_str = str(row["X"]).replace(",", ".")

    try:
      lat = float(lat_str)
      lng = float(lng_str)
    except ValueError as e:
      print(f"\n✗ Error converting coordinates: {lat_str}, {lng_str} - {e}")
      print(f"  Original values: Y={row['Y']}, X={row['X']}")

    print(f"The coordinates of the CSV are {lng} and {lat}")
    try:
      data = get_beaches(lat, lng, radius)

      if data and "results" in data:
        valid_beaches = [p for p in data["results"] if is_valid_beach(p)]

        if valid_beaches:
          print(f"  ✓ Found {len(valid_beaches)} valid beach(es) out of {len(data['results'])} total places:")

          for beach in valid_beaches:
            beach_info = extract_beach_info(beach)
            beach_info["Search_X"] = lng
            beach_info["Search_Y"] = lat
            all_beaches.append(beach_info)
            beaches_found += 1

            print(f"    {beaches_found}. {beach_info.get('name', 'Unnamed beach')}")



        processed_locations +=1
        print(processed_locations)

        # Handle pagination (next page token) if needed
        if "next_page_token" in data:
            # Note: You'd need to handle pagination here
            # This requires additional API calls
            pass

      else:
        print(f"No results or error for location{lat},{lng}")

    except Exception as e:
      print(f"Error in processing {lat},{lng}: {e}")

    # Delay to respect API rate limits
    time.sleep(delay)


  return all_beaches

In [ ]:
all_beaches_data = process_coordinates(coordinates_df= coordinates)

processing..  122 coordinates


  0%|          | 0/122 [00:00<?, ?it/s]

The coordinates of the CSV are 13.53734493 and 38.08036623
  ✓ Found 2 valid beach(es) out of 21 total places:
    1. Spiaggia Aciddara
    2. Lungomare Porticello
1


  1%|          | 1/122 [00:04<08:26,  4.19s/it]

The coordinates of the CSV are 14.35122371 and 40.63355879
  ✓ Found 8 valid beach(es) out of 23 total places:
    3. Spiaggia di Sorrento
    4. Puolo Beach
    5. Spiaggia Pubblica Sorrento
    6. Bagni Salvatore
    7. Spiaggia rocciosa
    8. Spiaggia di Puolo
    9. Playa acceso publico
    10. Veniva dal mare Lucio Dalla
2


...

 98%|█████████▊| 120/122 [08:36<00:08,  4.30s/it]

The coordinates of the CSV are 13.36852026 and 38.16625971
  ✓ Found 2 valid beach(es) out of 15 total places:
    337. Spiaggia Vergine Maria
    338. Parcheggio attiguo Spiaggia Vergine Maria Palermo
121


 99%|█████████▉| 121/122 [08:40<00:04,  4.30s/it]

The coordinates of the CSV are 15.89431357 and 41.58052493
122


100%|██████████| 122/122 [08:44<00:00,  4.30s/it]


In [ ]:
# Get all beaches from Google
all_beaches_data = process_coordinates(coordinates_df=coordinates)

print(f"\nFound {len(all_beaches_data)} beaches from Google")

if len(all_beaches_data) > 0:
    # Create a list for matched results
    matched_beaches = []

    # For each beach in your CSV, find the best match
    for idx, row in beaches_df.iterrows():
        csv_name = row["Beach"]
        csv_lat = row["lat"]
        csv_lng = row["lng"]

        best_match = None
        closest_distance = 999999
        match_type = "NO MATCH"

        # Look through Google results
        for google_beach in all_beaches_data:
            # Calculate simple distance in km
            distance = ((google_beach["lat"] - csv_lat)**2 +
                       (google_beach["lng"] - csv_lng)**2)**0.5 * 111

            # Check names
            csv_clean = clean_name(csv_name)
            google_clean = clean_name(google_beach["name"])

            # Priority 1: Exact or partial name match within 2km
            if distance < 1:
                if csv_clean == google_clean:
                    best_match = google_beach
                    match_type = "EXACT_NAME"
                    break
                elif csv_clean in google_clean or google_clean in csv_clean:
                    best_match = google_beach
                    match_type = "PARTIAL_NAME"
                    break
                elif distance < closest_distance:
                    closest_distance = distance
                    best_match = google_beach
                    match_type = "CLOSEST"

        # Create the record in your desired format
        if best_match:
            matched_beaches.append({
                "Beach_Code": idx + 1,  # Sequential number
                "Original_Beach_Name": csv_name,
                "API_Beach_Name": best_match["name"],
                "Match_Type": match_type,
                "rating": best_match["rating"],
                "user_ratings_total": best_match["user_ratings_total"],
                "lat": best_match["lat"],
                "lon": best_match["lng"],
                "Search_lat": csv_lat,
                "Search_lng": csv_lng,

            })
        else:
            matched_beaches.append({
                "Beach_Code": idx + 1,
                "Original_Beach_Name": csv_name,
                "API_Beach_Name": "NOT FOUND",
                "Match_Type": "NO MATCH",
                "rating": None,
                "user_ratings_total": None,
                "lat": None,
                "lon": None,
                "Search_lat": csv_lat,
                "Search_lng": csv_lng,
                
            })

    # Save to CSV in your exact format
    matched_df = pd.DataFrame(matched_beaches)

    # Reorder columns to match the format exactly
    columns_order = [
        "Beach_Code", "Original_Beach_Name", "API_Beach_Name", "Match_Type",
        "rating", "user_ratings_total", "API_lat", "API_lng",
        "lat", "lon",
    ]
    matched_df = matched_df[columns_order]

    # Save the file
    matched_df.to_csv("beaches_v2-b.csv", index=False)

    print(f"\n✅ Saved {len(matched_df)}'")


else:
    print("\n❌ No beaches found from Google API")

processing..  122 coordinates


  0%|          | 0/122 [00:00<?, ?it/s]

The coordinates of the CSV are 13.53734493 and 38.08036623
  ✓ Found 2 valid beach(es) out of 21 total places:
    1. Spiaggia Aciddara
    2. Lungomare Porticello
1


  1%|          | 1/122 [00:04<08:50,  4.38s/it]

The coordinates of the CSV are 14.35122371 and 40.63355879
  ✓ Found 8 valid beach(es) out of 23 total places:
    3. Spiaggia di Sorrento
    4. Puolo Beach
    5. Spiaggia Pubblica Sorrento
    6. Bagni Salvatore
    7. Spiaggia rocciosa
    8. Spiaggia di Puolo
    9. Playa acceso publico
    10. Veniva dal mare Lucio Dalla
2


...

The coordinates of the CSV are 13.36852026 and 38.16625971
  ✓ Found 2 valid beach(es) out of 15 total places:
    336. Spiaggia Vergine Maria
    337. Parcheggio attiguo Spiaggia Vergine Maria Palermo
121


 99%|█████████▉| 121/122 [08:43<00:04,  4.31s/it]

The coordinates of the CSV are 15.89431357 and 41.58052493
122


100%|██████████| 122/122 [08:47<00:00,  4.32s/it]



Found 337 beaches from Google

✅ Saved 122 matched beaches to 'Matched_Beaches_Results.csv'

First 5 results:
   Beach_Code                          Original_Beach_Name  \
0           1                                     Aciddara   
1           2  Bagni della Regina Giovanna - Sorrento (Na)   
2           3                            baia delle favole   
3           4                 Boschetto Steccato di Cutro    
4           5                             Cala Santa Maria   

                      API_Beach_Name    Match_Type  rating  \
0                  Spiaggia Aciddara  PARTIAL_NAME     3.5   
1                  Spiaggia rocciosa       CLOSEST     4.5   
2                  Baia delle Favole    EXACT_NAME     4.5   
3  Spiaggia Libera Steccato di Cutro       CLOSEST     4.4   
4             Torre Cala Santa Maria  PARTIAL_NAME     4.0   

   user_ratings_total    API_lat    API_lng  Search_lat  Search_lng  \
0               357.0  38.080293  13.537561   38.080366   13.537345   
1

In [ ]:
# Check the column names in your beaches_df
print("Columns in beaches_df:", beaches_df.columns.tolist())
print("\nFirst few rows of beaches_df:")
print(beaches_df.head(10))

Columns in beaches_df: ['Beach code', 'Beach', 'X', 'Y', 'Orientation (Codice)', 'Orientation', 'Region', 'Country', 'lat', 'lng', 'clean_beach_name']

First few rows of beaches_df:
   Beach code                                        Beach            X  \
0          40                                     Aciddara  13,53734493   
1         149  Bagni della Regina Giovanna - Sorrento (Na)  14,35122371   
2          59                            baia delle favole  9,391254298   
3         123                 Boschetto Steccato di Cutro   16,93195671   
4         105                             Cala Santa Maria  13,19533405   
5         142                              Caletta Caretta  17,77187437   
6         102                         Caletta di Sant'Elia  13,53882176   
7          46                      Capo Arena- budda beach   15,7836628   
8         139                                  Capocolonna  17,20197201   
9          98                                   Capparrina   12,9717

**What went good**

This jupyter notebook file extracts successfully from Google Places the User Rating and User Reviews using the API key. The extracted beaches are then matched by their names and their location making able to creating csv file that contains only the beaches our report_eng(Beaches) and get information only for them.

---
**What went bad**

The parameter settings for calling Google Place API has limited options as result it returns names for the beaches that can be tourist attractions, establishments etc.

---

**What could have done better**

The optimization of the script to get the beaches that matches with beaches from the Google Places. Also to load a dataset containing beach locations and names that directly match those from Google Places, in order to retrieve the correct beach from Google Places